<a href="https://colab.research.google.com/github/githubpsyche/rememberly/blob/main/TextSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 1.8MB 26.1MB/s 
     |████████████████████████████████| 1.2MB 49.7MB/s 
     |████████████████████████████████| 890kB 53.5MB/s 
     |████████████████████████████████| 3.2MB 59.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=3eb713a28030f552f0aba80a65939107c383dfa9a24c99c5bdbf71c072dc6ead
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a277919346fb63e6b8542cfa9035eccab3228ea85559c4094a298ae31c5f4b39
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
# imports
import torch
import torch.nn.functional as F

import scipy.spatial
from sentence_transformers import SentenceTransformer

# Testing

In [5]:
model = SentenceTransformer("stsb-roberta-base")

100%|██████████| 461M/461M [00:26<00:00, 17.2MB/s]


In [6]:
model = model.to(dev)

In [7]:
# testing spacy
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

In [16]:
embeds = model.encode(sentences, convert_to_tensor=True)

In [13]:
type(embeds)

torch.Tensor

In [18]:
distance.pdist(embeds, metric="cosine")

array([0.50537506, 0.96602266, 0.97348109])

In [33]:
F.cosine_similarity(embeds[2:3], embeds[1:2])

tensor([0.0265])

In [59]:
text_units = [["this is one", "of the many", "text units", "we have", "to use"],
              ["this is another"],
              ["do you think", "this could be", "a third?"]]

In [63]:
text_unit_embeds = [model.encode(i, convert_to_tensor=True) for i in text_units]

In [65]:
[distance.pdist(i, metric="cosine") for i in text_unit_embeds]

[array([0.66904016, 0.7714568 , 0.3554559 , 0.61600814, 0.85547665,
        0.498224  , 0.68915933, 0.94102719, 0.66680099, 0.66650351]),
 array([], dtype=float64),
 array([0.72241644, 0.63780705, 0.84130491])]

# Semantic Text Similarity

In [3]:
# doing something like this is important for speed but with GCP something else might
# have to be done
if torch.cuda.is_available():
  dev = torch.device("cuda:0")
else:
  dev = torch.device("cpu")

Note: SentenceTransformer("sbert_model_name") will download and store the sbert model we choose each session we build a TextSimilarity class. This takes a while, but once a certain model is downloaded, it's easy to use with multiple instances of TextSimilarity

Also note that SBERT takes a while to run since it's fairly big, and pdist is O(n^2), this will become a lot slower with longer reading cycles. On the order of sentences as reading cycles, I don't think it will be too bad, but this is also on the Colab GPU

In [6]:
# export
class TextSimilarity(torch.nn.Module):
  """
  Computes embeddings for a pair of given sentences and calculates the cosine
  similarity between them
  """
  def __init__(self, sbert_model_name):
    """
    sbert_model_name: sbert model to use, I think "stsb-roberta-base" is good
    """
    super(TextSimilarity, self).__init__()
    self.model = SentenceTransformer(sbert_model_name)
    self.to(dev)

  @staticmethod
  def measure(embedded_reading_cycle):
    """
    Computes cosine similarity between all text unit embeddings in a reading cycle
    reading_cycle: list of text unit embeddings
    """
    # no customizable metric - only cosine, otherwise 1 - x doesn't make sense
    similarities = 1 - scipy.spatial.distance.pdist(embedded_reading_cycle, metric="cosine")
    # convert to torch tensors
    return torch.tensor(similarities)

  def forward(self, reading_cycles):
    """
    Computes cosine similarities between n sentences as a matrix
    reading_cycles: list of lists of text unit strings
    """
    # embeds is the list of text_unit embeddings for each reading cycle of shape
    # (num_reading_cycles, num_text_units, embed_dim)
    # where num_text_units varies per reading cycle, and embed_dim = 768 as per BERT
    embeds = [self.model.encode(reading_cycle, convert_to_tensor=True) for reading_cycle in reading_cycles]
    # given n text units in a given reading_cycle
    # pdist will compute n * (n - 1) // 2 similarities, so measures is a list of shape
    # of shape (num_reading_cycles, num_text_units * (num_text_units - 1) // 2)
    measures = [self.measure(emb_reading_cycle) for emb_reading_cycle in embeds]
    # return similarity measures and embeddings
    return measures, embeds

In [7]:
# testing
ts = TextSimilarity("stsb-roberta-base") # takes a while to load
print(ts([["this is one", "of the many", "text units", "we have", "to use"],
              ["this is another"],
              ["do you think", "this could be", "a third?"]])[0])

[tensor([0.3310, 0.2285, 0.6445, 0.3840, 0.1445, 0.5018, 0.3108, 0.0590, 0.3332,
        0.3335], dtype=torch.float64), tensor([], dtype=torch.float64), tensor([0.2776, 0.3622, 0.1587], dtype=torch.float64)]
